In [1]:
import pandas as pd
import numpy as np
import glob
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix
import os
from sklearn.neighbors import NearestNeighbors

## Loading the data

In [2]:
def load_data(data_dir):
    """
    Load all the data from the data directory
    """
    data = []
    for file in glob.glob(os.path.join(data_dir, '*.csv')):
        data.append(pd.read_csv(file))
    return pd.concat(data, ignore_index=True)

In [3]:
path_to_train = r'C:\Users\marce\Desktop\PosTech\datathon\challenge-webmedia-e-globo-2023\files\treino\treino_parte1.csv'

In [4]:
train_df = pd.read_csv(path_to_train)
train_df.head()

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,"c8aab885-433d-4e46-8066-479f40ba7fb2, 68d2039c...","1657146417045, 1657146605778, 1657146698738","76, 38, 41","20380, 21184, 35438","50.3, 18.18, 16.46","2, 1, 1","1657146417045, 1657146605778, 1657146698738"
1,2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...,Non-Logged,60,"3325b5a1-979a-4cb3-82b6-63905c9edbe8, fe856057...","1656684240278, 1656761266729, 1656761528085, 1...","7, 80, 2, 1, 7, 62, 26, 44, 4, 4, 14, 45, 13, ...","6049, 210489, 8672, 10000, 30000, 123007, 9965...","25.35, 45.66, 35.3, 28.05, 36.53, 47.57, 55.33...","1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1...","1656684240278, 1656761266729, 1656761528085, 1..."
2,0adffd7450d3b9840d8c6215f0569ad942e782fb19b805...,Logged,107,"04756569-593e-4133-a95a-83d35d43dbbd, 29b6b142...","1656678946256, 1656701076495, 1656701882565, 1...","0, 0, 0, 0, 0, 44, 0, 0, 2, 1, 0, 0, 0, 44, 0,...","311274, 140000, 32515, 157018, 118689, 159243,...","67.58, 47.22, 41.52, 63.09, 51.38, 65.11, 71.9...","1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1...","1656678946256, 1656701076495, 1656701882565, 1..."
3,c1e8d644329a78ea1f994292db624c57980b2886cfbc2d...,Non-Logged,56,"1f2b9c2f-a2d2-4192-b009-09065da8ec23, 04756569...","1658333312180, 1658404553818, 1658408449062, 1...","8, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 1, 1...","182696, 91925, 30000, 273655, 126409, 42980, 1...","58.26, 72.66, 22.57, 59.89, 40.36, 36.35, 14.7...","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...","1658333312180, 1658404553818, 1658408449062, 1..."
4,e777d1f31d4d955b63d60acc13df336d3903f52ab8f8f4...,Non-Logged,4,"bebdeb3e-1699-43e0-a1b8-989f5a6ab679, f4b484a7...","1658766608801, 1658766608801, 1660084035094, 1...","579, 579, 7, 2","801396, 801396, 10000, 10000","78.74, 78.74, 16.71, 9.34","7, 7, 1, 1","1658766608801, 1658766608801, 1660084035094, 1..."


In [5]:
# Definir caminhos para os arquivos de treino (supondo que estejam nomeados como treino_parte_*.csv)
path_to_items_folder = r"C:\Users\marce\Desktop\PosTech\datathon\challenge-webmedia-e-globo-2023\itens\itens"

In [6]:
items_df = load_data(path_to_items_folder)
items_df.head()

,page,url,issued,modified,title,body,caption
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém\nÁdrio Denner/ AD Prod...,As expressões santarenas não significam apenas...
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,2023-04-15 04:25:39+00:00,Ex-premiê Shinzo Abe morre após ser baleado no...,Novo vídeo mostra que assassino de Shinzo Abe ...,Ex-primeiro-ministro foi atingido por tiros de...
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,2023-06-07 17:44:54+00:00,"Relator no STF, Fachin vota contra marco tempo...","Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...
4,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,2023-06-07 17:43:39+00:00,"\nApós 2 votos, pedido de vista suspende julga...",Após um pedido de vista (mais tempo para análi...,"Pelo marco temporal, índios só podem reivindic..."


In [7]:
# Lista de colunas que devem ser convertidas para listas
colunas_listas = [
    'history',
    'timestampHistory',
    'numberOfClicksHistory',
    'pageVisitsCountHistory',
    'timestampHistory_new',
    'scrollPercentageHistory',
    'timeOnPageHistory'
    
]

In [8]:
def converter_str_para_lista(df, colunas):
    for col in colunas:
        # Verifica se a coluna existe e se o valor não é nulo
        df[col] = df[col].apply(lambda x: x.split(", ") if pd.notnull(x) else [])
    return df

In [9]:
# Aplicar a conversão
train_df = converter_str_para_lista(train_df, colunas_listas)

In [10]:
# Exemplo: Verificando a coluna 'history' para um usuário específico
user_id = "2c1080975e257ed630e26679edbe4d5c850c65f3e09f655798b0bba9b42f2110"
print("Histórico do usuário:", train_df.loc[train_df['userId'] == user_id, 'history'].iloc[0])

Histórico do usuário: ['3325b5a1-979a-4cb3-82b6-63905c9edbe8', 'fe856057-f97d-419f-ab1c-97c5c3e0719c', 'd1da8d3f-bf30-4776-8d75-5f4e24cc134e', '8d7b7c79-4a54-4d5f-8d02-36805d60c498', '982578ac-ce37-4330-aec8-b44843e79f67', '083220a2-455c-4b64-b47a-807af2d6f00a', 'e752d7a2-d87b-4894-8c91-9743cc644b77', '96a30f4a-a9b2-44fd-89ae-4fd14e5cf246', '9a9b6a92-4a97-4c4d-99df-16fbe9546f0a', 'f03b8e40-56dd-410c-a04e-2711df683287', 'b409a984-c60d-4852-a55f-8e153ce9e69a', 'a1fb4a4d-4ad6-4dfe-9567-bceff5113d0b', '99d60eed-cda1-48fd-8349-00494ed9dd46', 'c511189f-5ced-4b0b-a06f-fcdb17264c24', 'c725a9d0-c658-4431-bbe9-5f592b56b990', '2d63c994-b72e-472b-ba73-c978916a6015', 'e1880fc6-c9dc-4565-b378-952f04e6a4d4', '370d60b4-7936-4239-9745-f1ecac5a1bbc', '5133418b-211f-4f58-9d25-86b46addfbc8', 'bbd29779-404b-4c34-8c4e-cddc0852d5b0', '21ac2572-76f3-4539-9369-bd7b629fd5dd', '179a4335-cc5b-4728-9a6c-a1d0a37a262c', '8bb97465-965f-4910-ac62-9d17af26b2bf', '5b902915-fab0-48a1-913f-3b12fe047372', 'ef5ec9cb-de05-4a

In [11]:
# Converter IDs para string, se necessário
train_df['userId'] = train_df['userId'].astype(str)

In [12]:
train_df.head()

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,"[c8aab885-433d-4e46-8066-479f40ba7fb2, 68d2039...","[1657146417045, 1657146605778, 1657146698738]","[76, 38, 41]","[20380, 21184, 35438]","[50.3, 18.18, 16.46]","[2, 1, 1]","[1657146417045, 1657146605778, 1657146698738]"
1,2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...,Non-Logged,60,"[3325b5a1-979a-4cb3-82b6-63905c9edbe8, fe85605...","[1656684240278, 1656761266729, 1656761528085, ...","[7, 80, 2, 1, 7, 62, 26, 44, 4, 4, 14, 45, 13,...","[6049, 210489, 8672, 10000, 30000, 123007, 996...","[25.35, 45.66, 35.3, 28.05, 36.53, 47.57, 55.3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, ...","[1656684240278, 1656761266729, 1656761528085, ..."
2,0adffd7450d3b9840d8c6215f0569ad942e782fb19b805...,Logged,107,"[04756569-593e-4133-a95a-83d35d43dbbd, 29b6b14...","[1656678946256, 1656701076495, 1656701882565, ...","[0, 0, 0, 0, 0, 44, 0, 0, 2, 1, 0, 0, 0, 44, 0...","[311274, 140000, 32515, 157018, 118689, 159243...","[67.58, 47.22, 41.52, 63.09, 51.38, 65.11, 71....","[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, ...","[1656678946256, 1656701076495, 1656701882565, ..."
3,c1e8d644329a78ea1f994292db624c57980b2886cfbc2d...,Non-Logged,56,"[1f2b9c2f-a2d2-4192-b009-09065da8ec23, 0475656...","[1658333312180, 1658404553818, 1658408449062, ...","[8, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 1, ...","[182696, 91925, 30000, 273655, 126409, 42980, ...","[58.26, 72.66, 22.57, 59.89, 40.36, 36.35, 14....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1658333312180, 1658404553818, 1658408449062, ..."
4,e777d1f31d4d955b63d60acc13df336d3903f52ab8f8f4...,Non-Logged,4,"[bebdeb3e-1699-43e0-a1b8-989f5a6ab679, f4b484a...","[1658766608801, 1658766608801, 1660084035094, ...","[579, 579, 7, 2]","[801396, 801396, 10000, 10000]","[78.74, 78.74, 16.71, 9.34]","[7, 7, 1, 1]","[1658766608801, 1658766608801, 1660084035094, ..."


In [13]:
# Função para "explodir" as interações: gera uma linha para cada item no histórico do usuário
def explodir_interacoes(df):
    registros = []
    for idx, row in df.iterrows():
        user_id = row['userId']
        user_type = row['userType']
        history = row['history']
        timestamps = row['timestampHistory_new']
        clicks = row['numberOfClicksHistory']
        visits = row['pageVisitsCountHistory']
        time_on_page = row['timeOnPageHistory']
        
        # Para cada item no histórico, cria um registro único
        for i in range(len(history)):
            reg = {
                'userId': user_id,
                'userType': user_type,
                'itemId': history[i],
                'timestamp': timestamps[i],
                'clicks': clicks[i],
                'visits': visits[i],
                'timeOnPage': time_on_page[i]
            }
            registros.append(reg)
    df_interacoes = pd.DataFrame(registros)
    return df_interacoes

In [14]:
# Aplicar a função ao dataframe de treino (train_df já deve ter as colunas convertidas para listas)
df_interacoes = explodir_interacoes(train_df)
print("Exemplo de interações:")
print(df_interacoes.head())

Exemplo de interações:
                                              userId    userType  \
0  f98d1132f60d46883ce49583257104d15ce723b3bbda21...  Non-Logged   
1  f98d1132f60d46883ce49583257104d15ce723b3bbda21...  Non-Logged   
2  f98d1132f60d46883ce49583257104d15ce723b3bbda21...  Non-Logged   
3  2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...  Non-Logged   
4  2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...  Non-Logged   

                                 itemId      timestamp clicks visits  \
0  c8aab885-433d-4e46-8066-479f40ba7fb2  1657146417045     76      2   
1  68d2039c-c9aa-456c-ac33-9b2e8677fba7  1657146605778     38      1   
2  13e423ce-1d69-4c78-bc18-e8c8f7271964  1657146698738     41      1   
3  3325b5a1-979a-4cb3-82b6-63905c9edbe8  1656684240278      7      1   
4  fe856057-f97d-419f-ab1c-97c5c3e0719c  1656761266729     80      1   

  timeOnPage  
0      20380  
1      21184  
2      35438  
3       6049  
4     210489  


In [15]:
df_interacoes['timestamp'] = pd.to_datetime(df_interacoes['timestamp'], unit='ms')

C:\Users\marce\AppData\Local\Temp\ipykernel_6952\2590334139.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_interacoes['timestamp'] = pd.to_datetime(df_interacoes['timestamp'], unit='ms')


In [16]:
# df_interacoes.to_csv(r'C:\Users\marce\Desktop\PosTech\datathon\challenge-webmedia-e-globo-2023\interacoes\interacoes.csv', index=False)

In [17]:
columns_to_transform = ['visits', 'timeOnPage']

In [18]:
df_interacoes[columns_to_transform] = df_interacoes[columns_to_transform].astype(int)

In [19]:
df_top_news = df_interacoes.groupby('itemId').agg({
    'visits': 'sum', 
    'timeOnPage': 'sum'
}).reset_index()

In [20]:
df_top_news = df_top_news.merge(items_df[['page', 'issued']], left_on='itemId', right_on='page', how='left')
df_top_news.drop(columns=['page'], inplace=True)

In [21]:
# df_top_news.to_csv(r'C:\Users\marce\Desktop\PosTech\datathon\challenge-webmedia-e-globo-2023\interacoes\top_news.csv', index=False)

In [22]:
# %% (após a criação de df_interacoes e a conversão de timestamp para numérico)
df_interacoes['timestamp'] = pd.to_datetime(df_interacoes['timestamp'], unit='ms')
df_interacoes['timestamp_numeric'] = df_interacoes['timestamp'].apply(lambda x: x.timestamp())

In [23]:
# %% Cálculo da recency usando decaimento exponencial
# Utilizando o maior timestamp como referência (você pode usar pd.Timestamp.now().timestamp() se preferir)
current_time = df_interacoes['timestamp_numeric'].max()
lambda_decay = 0.00000000001  # Parâmetro de decaimento; ajuste conforme necessário
df_interacoes['recency'] = np.exp(-lambda_decay * (current_time - df_interacoes['timestamp_numeric']))

In [24]:
# %% Converter a coluna 'clicks' para numérico e aplicar transformação logarítmica
df_interacoes['clicks'] = pd.to_numeric(df_interacoes['clicks'], errors='coerce')
df_interacoes['clicks_log'] = np.log1p(df_interacoes['clicks'])

In [25]:
# %% Converter a coluna 'clicks' para numérico e aplicar transformação logarítmica
df_interacoes['recency'] = pd.to_numeric(df_interacoes['recency'], errors='coerce')
df_interacoes['recency_log'] = np.log1p(df_interacoes['recency'])

In [26]:
# %% Converter a coluna 'clicks' para numérico e aplicar transformação logarítmica
df_interacoes['timeOnPage'] = pd.to_numeric(df_interacoes['timeOnPage'], errors='coerce')
df_interacoes['timeOnPage_log'] = np.log1p(df_interacoes['timeOnPage'])

In [27]:
# %% Normalização dos campos 'clicks_log' e 'timeOnPage'
features = ['clicks_log', 'timeOnPage_log', 'recency']
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(df_interacoes[features])
df_scaled = pd.DataFrame(scaled_values, columns=[f"{col}_scaled" for col in features])
df_interacoes = pd.concat([df_interacoes, df_scaled], axis=1)

In [28]:
# %% Definir os pesos para cada métrica
w_recency      = 10  # Dá maior importância às interações recentes
w_clicks       = 0.5
w_time_on_page = 1.0

In [29]:
# %% Calcular o score final combinando recency, cliques e tempo na página
df_interacoes['score'] = (
    w_recency      * df_interacoes['recency_scaled'] +
    w_clicks       * df_interacoes['clicks_log_scaled'] +
    w_time_on_page * df_interacoes['timeOnPage_log_scaled']
)

In [30]:
unique_items = df_interacoes['itemId'].unique()
item_id_to_index = {item: idx for idx, item in enumerate(unique_items)}

In [31]:
# Gerar os mapeamentos a partir do df_interacoes
unique_users = df_interacoes['userId'].unique()
user_id_to_index = {user: idx for idx, user in enumerate(unique_users)}

In [32]:
# Construir a matriz esparsa de interações
# Cada linha representa um usuário e cada coluna um item; o valor é a contagem de interações
rows = df_interacoes['userId'].map(user_id_to_index).values
cols = df_interacoes['itemId'].map(item_id_to_index).values
data = df_interacoes['score'].values  # Agora cada interação tem um peso diferente
# data = np.ones(len(df_interacoes))  # Peso 1 para cada interação

In [33]:
interaction_matrix = csr_matrix((data, (rows, cols)), shape=(len(unique_users), len(unique_items)))
print("Matriz de interação (esparsa):", interaction_matrix.shape)

Matriz de interação (esparsa): (100000, 108573)


## Collaborative filtering Model

In [34]:
# Definir o número de vizinhos para a busca (valor ajustável conforme a necessidade)
n_neighbors = 100

In [35]:
# Criar o modelo de vizinhos para o CF, utilizando distância cosseno (com "brute" para matrizes esparsas)
nn_model_cf = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=n_neighbors, n_jobs=-1)
nn_model_cf.fit(interaction_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=100)

In [91]:
def recommend_popular_items(top_n=10, weight_recency=0.3, weight_visits=0.4, weight_time=0.3):
    """
    Calcula um score de popularidade para cada notícia com base na data de lançamento (Issued),
    número de visitas e tempo na página.
    
    Parâmetros:
    - df: DataFrame contendo as colunas 'Issued', 'visits' e 'timeOnPage'.
    - top_n: quantidade de notícias mais populares a serem retornadas.
    - weight_recency: peso para o fator de recência.
    - weight_visits: peso para o número de visitas.
    - weight_time: peso para o tempo na página.
    
    Retorna:
    - DataFrame com as top_n notícias ordenadas pela popularidade.
    """

    # carregar df_top_news
    df = df_top_news

    # Converter a coluna 'Issued' para datetime, se ainda não estiver
    df['issued'] = pd.to_datetime(df['issued'], utc=True)
    
    # Usar a data atual ou a data máxima do DataFrame como referência para recência
    reference_date = pd.Timestamp.now(tz='UTC')  # ou: df['Issued'].max()
    df['days_since'] = (reference_date - df['issued']).dt.days
    
    # Calcular um score de recência: quanto menor os dias desde a emissão, maior o score
    # Usamos 1 / (dias + 1) para evitar divisão por zero
    df['recency_score'] = 1 / (df['days_since'] + 1)
    
    # Normalizar as colunas 'visits' e 'timeOnPage' para o intervalo [0,1]
    df['visits_norm'] = (df['visits'] - df['visits'].min()) / (df['visits'].max() - df['visits'].min() + 1e-9)
    df['timeOnPage_norm'] = (df['timeOnPage'] - df['timeOnPage'].min()) / (df['timeOnPage'].max() - df['timeOnPage'].min() + 1e-9)
    
    # Calcular o score de popularidade combinando as métricas com os pesos definidos
    df['popularity_score'] = (weight_recency * df['recency_score'] +
                              weight_visits * df['visits_norm'] +
                              weight_time * df['timeOnPage_norm'])
    
    # Ordenar o DataFrame com base no score de popularidade em ordem decrescente
    df_popular = df.sort_values('popularity_score', ascending=False)
    
    return df_popular[:top_n]


In [95]:
# Exemplo de uso:
# Suponha que seu DataFrame 'top_news_df' contenha as colunas 'Issued', 'visits' e 'timeOnPage'
df_top = calcular_popularidade( top_n=5)
print(df_top[['issued', 'itemId', 'visits', 'timeOnPage', 'popularity_score']])

                          issued                                itemId  \
88815  2022-07-11 03:20:57+00:00  d2593c3d-2347-40d9-948c-b6065e8459a9   
104591 2022-07-20 11:14:32+00:00  f6b5d170-48b9-4f8e-88d4-c84b6668f3bd   
13026  2022-08-10 09:55:29+00:00  1f32787b-de2b-49be-8c20-ddaeae34cc22   
102063 2022-08-03 20:08:46+00:00  f0a78e58-ec7e-494c-9462-fbd6446a9a89   
44752  2022-07-27 13:54:29+00:00  6a83890a-d9e9-4f6b-a6c6-90d031785bbf   

        visits  timeOnPage  popularity_score  
88815     4650   380807037          0.700312  
104591    4242   344092884          0.636287  
13026     3782   309536050          0.569491  
102063    3560   319667919          0.558370  
44752     3327   290357558          0.515229  


In [102]:
def cf_recommend(user_id, top_n=10, n_neighbors=n_neighbors):
    """
    Recomenda itens para um usuário utilizando vizinhos mais próximos (CF via NearestNeighbors).
    Para um dado usuário, busca os n vizinhos mais próximos com base na interação,
    acumula os itens que eles consumiram (excluindo os já consumidos pelo usuário) e os ordena por similaridade agregada.
    """
    if user_id not in user_id_to_index:
        return list(recommend_popular_items(top_n)['itemId'])
    
    user_idx = user_id_to_index[user_id]
    
    # Buscar os vizinhos mais próximos para o usuário (a consulta retorna o próprio usuário com distância 0)
    distances, indices = nn_model_cf.kneighbors(interaction_matrix[user_idx], n_neighbors=n_neighbors)
    distances = distances.flatten()
    indices = indices.flatten()
    
    # Converter a distância em similaridade (1 - distância, pois a distância cosseno varia entre 0 e 1)
    # E acumular os itens dos vizinhos, ignorando os itens já consumidos pelo usuário
    user_items = set(interaction_matrix[user_idx].nonzero()[1])
    rec_scores = {}
    
    for neighbor_idx, dist in zip(indices, distances):
        # Ignorar o próprio usuário
        if neighbor_idx == user_idx:
            continue
        similarity = 1 - dist  # Similaridade: quanto menor a distância, maior a similaridade
        neighbor_items = interaction_matrix[neighbor_idx].nonzero()[1]
        for item in neighbor_items:
            if item in user_items:
                continue  # Ignorar itens já consumidos
            rec_scores[item] = rec_scores.get(item, 0) + similarity

    # Ordenar os itens por pontuação decrescente e retornar os top_n (convertendo os índices para IDs reais)
    ranked_items = sorted(rec_scores.items(), key=lambda x: x[1], reverse=True)
    recommended_item_ids = [unique_items[idx] for idx, score in ranked_items[:top_n]]
    
    return recommended_item_ids

In [104]:
# f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069
# Exemplo de recomendação CF para um usuário específico
user_id = "f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069"
print("CF Recommendation para o usuário {}: {}".format(user_id, cf_recommend(user_id, top_n=5)))

CF Recommendation para o usuário f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069: ['ad543dfb-2ddd-4878-9f6a-83fbefbdd826', '9475e585-876a-4efa-861c-cb1fe423018f', 'd98df164-1b93-4723-aceb-363a686ab9bc', 'ca0b2d4c-3c76-4327-a159-bf6d5893165e', 'a12a1004-93d7-4131-a8b9-89e4512d803b']


In [100]:
user_idx = user_id_to_index[user_id]
user_interactions = interaction_matrix[user_idx].nonzero()[1]
print("Itens já consumidos pelo usuário:", [unique_items[i] for i in user_interactions])


Itens já consumidos pelo usuário: ['c8aab885-433d-4e46-8066-479f40ba7fb2', '68d2039c-c9aa-456c-ac33-9b2e8677fba7', '13e423ce-1d69-4c78-bc18-e8c8f7271964']


In [39]:
history_user_id = train_df[train_df['userId'] == user_id]['history'].values[0]
print("Histórico do usuário:", history_user_id)

Histórico do usuário: ['1f2b9c2f-a2d2-4192-b009-09065da8ec23', '04756569-593e-4133-a95a-83d35d43dbbd', 'b03986b4-7887-45f2-8269-a66b9d73a361', '9958aafa-3a3a-416c-a2b9-5d96769bad83', '69990b31-a6f9-4db0-80c3-1d4733f9e619', 'd4303151-c778-461b-8207-2e443db4b255', '722623f2-349b-4e8b-a974-ad8995f747ff', '9930ad3b-a506-4129-8efd-20c573365d74', 'faac2a94-565a-4fad-9db6-3abbb6e39c8a', '6dfb5395-decf-4b1f-ae70-959bde7417e2', '09d074f5-0cc9-4c31-9bb4-bc1c0d315675', 'b4070b48-1df6-4e82-ab9d-f7547c8a6da7', 'f89f8706-ffd4-4d86-af65-b2d0236996f4', '197de720-5810-4e35-b50c-02a1692f0e5a', '39410577-0d94-4790-ac41-81362bc8c915', '5db25cd8-32e2-4502-9208-b62d0e13b015', '14a54c8d-3155-4045-83bc-ae01af95a034', 'bcb7755a-5046-4bd7-81cd-8ead61a27ba7', 'f0a78e58-ec7e-494c-9462-fbd6446a9a89', 'ab93d775-dc30-45a2-b885-9dfcc26fd154', '470ccd72-1478-4a82-9e75-ecb6f732772a', '73ed314a-a9f3-4b76-83f1-67063a4ca520', '4154f103-9e39-49f5-8982-cdf13c576928', '4e9c2825-ff13-41ca-8e91-edd848060d19', '8c246d2b-81bd-4c

In [40]:
print(train_df.loc[train_df['userId'] == user_id, 'history'])

3    [1f2b9c2f-a2d2-4192-b009-09065da8ec23, 0475656...
Name: history, dtype: object


## Modelo Content-Based com TF-IDF

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached regex-2024.11.6-cp311-cp311-win_amd64.whl.metadata (41 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp311-cp311-win_amd64.whl (274 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marce\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [43]:
# Obter a lista de stopwords em português
stopwords_pt = stopwords.words('portuguese')

In [44]:
# items_df['combined_text'] = items_df['title'].fillna('')

items_df['combined_text'] = (items_df['title'].fillna('') + " " +
                            #  items_df['body'].fillna('') + " " +
                             items_df['caption'].fillna(''))

# Inicializar o TfidfVectorizer com limite de features para economia de memória
tfidf = TfidfVectorizer(stop_words=stopwords_pt, max_features=1000)
tfidf_matrix = tfidf.fit_transform(items_df['combined_text'])

In [45]:
# Em vez de calcular a matriz de similaridade completa, usamos NearestNeighbors
nn_model = NearestNeighbors(metric='cosine', algorithm='brute')
nn_model.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [46]:
def get_similar_items(item_id, top_n=10):
    """
    Dado um ID de item, retorna os top_n itens mais similares utilizando TF-IDF e NearestNeighbors.
    """
    if item_id not in items_df.index:
        return []
    
    # Obter o índice correspondente no tfidf_matrix
    item_idx = items_df.index.get_loc(item_id)
    query_vector = tfidf_matrix[item_idx]
    
    # Buscar os vizinhos mais próximos (n_neighbors inclui o próprio item, por isso solicitamos top_n+1)
    distances, indices = nn_model.kneighbors(query_vector, n_neighbors=top_n+1)
    
    # Ignorar o primeiro resultado (o próprio item) e mapear para os IDs dos itens
    similar_indices = indices.flatten()[1:]
    similar_item_ids = [items_df.index[i] for i in similar_indices]
    
    return similar_item_ids

In [50]:
# Exemplo: buscar itens similares para um item específico
item_id_exemplo = items_df.index[181860]
print("Itens similares para o item {}: {}".format(item_id_exemplo, get_similar_items(item_id_exemplo, top_n=5)))

Itens similares para o item 181860: [4801, 182233, 140510, 47810, 50498]


In [52]:
items_df.iloc[181860]

page                          49ce5419-6ee5-45f5-9034-26c16b473df7
url              http://g1.globo.com/ce/ceara/noticia/2022/02/2...
issued                                   2022-02-26 19:21:10+00:00
modified                                 2022-02-26 19:21:11+00:00
title            Juazeiro do Norte tem a gasolina comum mais ba...
body             Juazeiro do Norte tem a gasolina comum mais ba...
caption          O município do Cariri registrou um preço mínim...
combined_text    Juazeiro do Norte tem a gasolina comum mais ba...
Name: 181860, dtype: object

In [53]:
print(items_df.iloc[181860]['title'])
print(items_df.iloc[4801]['title'])
print(items_df.iloc[182233]['title'])
print(items_df.iloc[140510]['title'])
print(items_df.iloc[47810]['title'])
print(items_df.iloc[50498]['title'])

Juazeiro do Norte tem a gasolina comum mais barata, e Itapipoca a mais cara, no Ceará, diz pesquisa da ANP
Petrobras reduz novamente preço da gasolina e diesel nas refinarias
Preço da gasolina chega a R$ 8,52 no interior do Ceará; veja valores por cidade
Agrotóxico pode gerar puberdade precoce e malformação de bebês no Ceará, diz pesquisadora
Preço médio da gasolina está R$ 0,88 mais barato em São José, aponta pesquisa do Procon
SC tem diferença no preço de combustíveis por região, aponta pesquisa


* 73027: [227748, 14414, 196714, 224865, 206553]
* 206553: [224865, 226625, 50498, 194649, 172758]
* >50498: [164851, 194649, 230525, 172758, 224865]
*   Manifestantes fazem ato contra o presidente Bolsonaro no Centro do Rio 
*   Manifestantes protestam no Centro do Rio contra o governo do presidente Jair Bolsonaro
*   Manifestantes fazem ato contra Bolsonaro no Centro do Rio
*   Manifestantes fazem ato contra Bolsonaro em São Luís
*   Manifestantes fazem ato em defesa da democracia e contra Bolsonaro em SP
*   Manifestantes se reúnem em ato pelo impeachment do presidente Jair Bolsonaro em Guararema

##  Abordagem Híbrida

In [ ]:
def hybrid_recommend(user_id, top_n=10, weight_cf=0.5, weight_cb=0.5):
    """
    Combina as recomendações de Collaborative Filtering (CF) e Content-Based (CB) usando KNN.
    Para cada usuário, são obtidas as recomendações CF e, para o histórico do usuário,
    utiliza-se o KNN para buscar itens similares. Os scores de ambos os métodos são
    combinados de forma ponderada para gerar o ranking final.
    """
    # Recomendações baseadas em CF
    recs_cf = cf_recommend(user_id, top_n=top_n*2)
    
    # Obter o histórico do usuário a partir do dataset de treino
    user_history = []
    df_user = train_df[train_df['userId'] == user_id]
    for hist in df_user['history']:
        user_history.extend(hist)
    user_history = list(set(user_history))
    
    # Recomendações baseadas em conteúdo utilizando KNN
    # Para cada item do histórico, buscamos os itens mais similares
    score_cb = {}
    for item in user_history:
        similar_items = get_similar_items(item, top_n=10)  # Obtemos os 10 itens mais similares para cada item
        for rec in similar_items:
            # Ignoramos itens já consumidos
            if rec in user_history:
                continue
            # Acumula uma contagem que pode representar a força do sinal
            score_cb[rec] = score_cb.get(rec, 0) + 1

    # Ordena os itens recomendados pelo conteúdo com base no score acumulado
    recs_cb = [item for item, score in sorted(score_cb.items(), key=lambda x: x[1], reverse=True)][:top_n*2]
    
    # Combinação simples: somamos os pesos das recomendações CF e CB
    score_dict = {}
    for item in recs_cf:
        score_dict[item] = score_dict.get(item, 0) + weight_cf
    for item in recs_cb:
        score_dict[item] = score_dict.get(item, 0) + weight_cb
        
    # Ordena os itens combinados e retorna os top_n
    ranked_items = sorted(score_dict.items(), key=lambda x: x[1], reverse=True)
    return [item for item, score in ranked_items][:top_n]


In [ ]:
# Exemplo de uso da recomendação híbrida:
print("Hybrid Recommendation para usuário {}: {}".format(user_id, hybrid_recommend(user_id=user_id, top_n=5)))

In [ ]:
items_df[items_df["page"] == "1f32787b-de2b-49be-8c20-ddaeae34cc22"][["title","caption"]]